In [3]:
from vgg16_places_365 import VGG16_Places365
base_model = VGG16_Places365(include_top=False, weights='places', input_shape=(150, 150, 3))
for layer in base_model.layers[:12]:
    layer.trainable = True
for layer in base_model.layers[12:]:
    layer.trainable = False

In [6]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)
 
test_datagen = ImageDataGenerator(rescale=1./255)
 
train_dir = './Places/train'
validation_dir = './Places/validation'
 
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 108x192
        target_size=(150, 150),
        batch_size=20,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')
 
 


Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [12]:
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.RMSprop(lr=2e-5),
            metrics=['acc'])
num_of_sample_train = 600
num_of_sample_validation = 150

batch_size = 20
history = model.fit_generator(
    train_generator,
    steps_per_epoch=num_of_sample_train/batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=num_of_sample_validation/batch_size)


Epoch 1/10
30/30 [==============================] - 174s 6s/step - loss: 1.0600 - acc: 0.6450 - val_loss: 0.4584 - val_acc: 0.9067
Epoch 2/10
30/30 [==============================] - 176s 6s/step - loss: 0.6595 - acc: 0.8333 - val_loss: 0.4478 - val_acc: 0.9333
Epoch 3/10
30/30 [==============================] - 176s 6s/step - loss: 0.5242 - acc: 0.9017 - val_loss: 0.2553 - val_acc: 0.9667
Epoch 4/10
30/30 [==============================] - 177s 6s/step - loss: 0.4502 - acc: 0.9367 - val_loss: 0.5771 - val_acc: 0.9600
Epoch 5/10
30/30 [==============================] - 176s 6s/step - loss: 0.3949 - acc: 0.9433 - val_loss: 0.7815 - val_acc: 0.9667
Epoch 6/10
30/30 [==============================] - 177s 6s/step - loss: 0.3734 - acc: 0.9517 - val_loss: 0.2854 - val_acc: 0.9733
Epoch 7/10
30/30 [==============================] - 180s 6s/step - loss: 0.3553 - acc: 0.9567 - val_loss: 0.4008 - val_acc: 0.9667
Epoch 8/10
30/30 [==============================] - 177s 6s/step - loss: 0.3280 - 

In [47]:
from keras.preprocessing import image
from keras.models import load_model
import keras

In [48]:
from keras import backend as K
def preprocess_input(x, dim_ordering='default'):
    if dim_ordering == 'default':
        dim_ordering = K.common.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}
    
    if dim_ordering == 'th':
        x[:, 0, :, :] -= 104.006
        x[:, 1, :, :] -= 116.669
        x[:, 2, :, :] -= 122.679
        x = x[:, ::-1, :, :]
    
    else:
        x[:, :, :, 0] -= 104.006
        x[:, :, :, 1] -= 116.669
        x[:, :, :, 2] -= 122.679
        x = x[:, :, :, ::-1]
    return x

In [57]:
import numpy as np
#Tagh_Bostan picture
ImageAddress = 'E:/2.jpg'
img = image.load_img(ImageAddress, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [58]:
prediction = model.predict(x)
y_classes = prediction.argmax()

In [59]:
labels = ['Azadi', 'Emam_Reza', 'Tagh_Bostan']

In [60]:
for i, p in enumerate(prediction[0]):
    print('%s Probability: \t %f' % (labels[i], p))

Azadi Probability: 	 0.000000
Emam_Reza Probability: 	 0.000000
Tagh_Bostan Probability: 	 1.000000
